<a href="https://colab.research.google.com/github/v-artur/Golden_Oreos/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#obtaining the data
!gdown https://drive.google.com/uc?id=1vtZchVzl424pSQBXQ8EBxvVOzcEQPKIp

#extracting it
import zipfile
zip_ref = zipfile.ZipFile("/content/features.zip", 'r')
zip_ref.extractall("/content/features")
zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1vtZchVzl424pSQBXQ8EBxvVOzcEQPKIp
To: /content/features.zip
100% 2.14G/2.14G [00:24<00:00, 88.2MB/s]


In [3]:
#setting the path
feat_path = r'/content/features'

# the subjects for each set
train_subs = ['sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09','sub-10']
val_subs = ['sub-01', 'sub-02']
test_subs = ['sub-03', 'sub-04']

In [6]:
from sklearn.decomposition import PCA
import numpy as np
import os


# making the train, validation and test sets with Principal Component Analysis 
def make_set(subs, n_comp):
  whole_data = []
  whole_spec = []

  for participant in subs:
    #features with PCA
    data = np.load(os.path.join(feat_path,f'{participant}_feat.npy'))
    transformer = PCA(n_components=n_comp, random_state=0)
    data = transformer.fit_transform(data)
    whole_data.append(data)
    #labels
    spec = np.load(os.path.join(feat_path,f'{participant}_spec.npy'))
    whole_spec.append(spec)
  
  features = np.vstack(tuple(whole_data))
  labels = np.vstack(tuple(whole_spec))

  return features, labels


train_data, train_spec = make_set(train_subs, 300)
val_data, val_spec = make_set(val_subs, 300)
test_data, test_spec = make_set(test_subs, 300)  

In [7]:
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
val_data = scaler.transform(val_data)
test_data = scaler.transform(test_data)

Simple FC-DNN model

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Input

#Making the Normal FC-DNN model

def create_dnn_model(inputsize, outputsize):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(inputsize)))
    model.add(tf.keras.layers.Dense(256, activation="relu", kernel_initializer='HeNormal'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(128, activation="relu", kernel_initializer='HeNormal'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(64, activation="relu", kernel_initializer='HeNormal'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(outputsize))
    return model


In [59]:
def concatenating_and_padding(subs):
  whole_data = []
  whole_spec = []

  for participant in subs:
    data = np.load(os.path.join(feat_path,f'{participant}_feat.npy'))
    whole_data.append(data)
    spec = np.load(os.path.join(feat_path,f'{participant}_spec.npy'))
    whole_spec.append(spec)

  new_spec = np.array(whole_spec)

  max_len = max([dat.shape[1] for dat in whole_data]) # max number of features
  all_len = sum([dat.shape[0] for dat in whole_data]) # sum of all rows

  new_data = np.zeros((all_len,max_len))

  num_rows = 0
  for sub in whole_data:
    row_len = sub.shape[1]
    new_data[num_rows:num_rows+sub.shape[0],0:row_len] = sub
    num_rows += sub.shape[0]

  return new_data, new_spec

In [8]:
#train_data = train_data[:700] # Experimenting on a smaller sample 
#train_spec = train_spec[:700]

In [45]:
# normal DNN model
early_stopping=EarlyStopping(patience=25, verbose=1, min_delta=1e-5)
checkpointer=ModelCheckpoint(filepath='weights1.hdf5', save_best_only=True, verbose=1)

model = create_dnn_model(300, 23)
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit(train_data, train_spec, batch_size=64, 
          epochs=100, verbose=0, validation_data=(val_data, val_spec), shuffle=True,
          callbacks=[checkpointer, early_stopping])

ValueError: ignored

## Model 1

In [ ]:
train_data = train_data.reshape((train_data.shape[0],train_data.shape[1],1))
train_spec = train_spec.reshape((train_spec.shape[0],train_spec.shape[1],1))

val_data = val_data.reshape((val_data.shape[0],val_data.shape[1],1))
val_spec = val_spec.reshape((val_spec.shape[0],val_spec.shape[1],1))

test_data = test_data.reshape((test_data.shape[0],test_data.shape[1],1))
test_spec = test_spec.reshape((test_spec.shape[0],test_spec.shape[1],1))

In [11]:
from keras.models import Model
from keras.layers import LSTM
from keras import Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [12]:
# https://www.kaggle.com/code/kmkarakaya/lstm-understanding-the-number-of-parameters

timesteps = train_data.shape[1]             # dimensionality of the input sequence
features = 1                          # dimensionality of each input representation in the sequence
outputDim = train_spec.shape[1]             # dimensionality of the LSTM outputs (Hidden & Cell states)

input = Input(shape=(timesteps, features))
output = LSTM(outputDim)(input)
model_LSTM = Model(inputs=input, outputs=output)

early_stopping=EarlyStopping(patience=25, verbose=1, min_delta=1e-5)
checkpointer = ModelCheckpoint(filepath='weights-lstm.hdf5', save_best_only=True, verbose=1)

model_LSTM.compile(loss='mse', optimizer='adam', metrics=['mse'])
model_LSTM.fit(train_data, train_spec, epochs=2, validation_split=0.2, batch_size=64, verbose=1, callbacks=[checkpointer, early_stopping])

Epoch 1/2
9/9 [==============================] - ETA: 0s - loss: 34.7186 - mse: 34.7186
Epoch 1: val_loss improved from inf to 21.96873, saving model to weights-lstm.hdf5
9/9 [==============================] - 4s 184ms/step - loss: 34.7186 - mse: 34.7186 - val_loss: 21.9687 - val_mse: 21.9687
Epoch 2/2
9/9 [==============================] - ETA: 0s - loss: 34.5797 - mse: 34.5797
Epoch 2: val_loss improved from 21.96873 to 21.81439, saving model to weights-lstm.hdf5
9/9 [==============================] - 1s 111ms/step - loss: 34.5797 - mse: 34.5797 - val_loss: 21.8144 - val_mse: 21.8144


In [13]:
x = test_data[0].reshape((1,300,1))
model_LSTM.predict(x)

1/1 [==============================] - 0s 471ms/step


array([[ 0.061514  , -0.01006135, -0.04190003,  0.00118558,  0.04889771,
         0.02200028, -0.02356197, -0.00060326,  0.01394669, -0.07210566,
        -0.00783062, -0.03763462,  0.1078046 ,  0.01875776, -0.00141495,
         0.00119135, -0.01632942,  0.07173061, -0.06324889,  0.01155341,
         0.09683105,  0.01851598,  0.03977415]], dtype=float32)

## Model 2

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [14]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

latent_dim = 128
start_symbol = 1.

#### ?
encoder_input_data = train_data
encoder_input_data[0,0,0] = start_symbol
decoder_input_data = train_spec
decoder_input_data[0,0,0] = start_symbol

start = np.zeros((1,train_spec.shape[1],1))
decoder_target_data = np.vstack((start, train_spec[:-1]))
####

encoder_inputs = Input(shape=(train_data.shape[1], 1))
encoder = LSTM(units=latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(train_spec.shape[1], 1))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(train_spec.shape[1], activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[decoder_outputs])

model.compile(optimizer='adam', loss='mse', metrics=['mse'])

early_stopping = EarlyStopping(patience=20, verbose=1, min_delta=1e-5)
checkpointer = ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
                    validation_split=0.2, batch_size=64, epochs=1, callbacks=[checkpointer, early_stopping]) # val?

9/9 [==============================] - ETA: 0s - loss: 34.2520 - mse: 34.2520
Epoch 1: val_loss improved from inf to 21.21002, saving model to weights.hdf5
9/9 [==============================] - 18s 1s/step - loss: 34.2520 - mse: 34.2520 - val_loss: 21.2100 - val_mse: 21.2100


In [15]:
# Inference:

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
# def decode_sequence2(input_seq):

#   states_value = encoder_model.predict(input_seq)
#   decoded_sentence = []

#   for t in range(23):
#     target_seq = np.zeros((1, 23, 1))
#     target_seq[0, t, 0] = 1.

#     outputs, h, c = decoder_model.predict(
#         [target_seq] + states_value)

#     decoded_sentence.append(np.argmax(outputs[0, -1, :])) # ?

#     states_value = [h, c]

#   return decoded_sentence

In [16]:
def decode_sequence(input_seq):
  input_seq[0,0,0] = start_symbol
  states_value = encoder_model.predict(input_seq)

  decoded_sentence = []
  target_seq = np.zeros((1, 23, 1))
  target_seq[0, 0, 0] = start_symbol

  for t in range(1,23):
    
    outputs, h, c = decoder_model.predict(
        [target_seq] + states_value)

    target_seq[0, t, 0] = np.argmax(outputs[0, -1, :])

    states_value = [h, c]

  return target_seq

In [17]:
x = test_data[0].reshape((1,300,1))
pred = decode_sequence(x)

1/1 [==============================] - 0s 26ms/step


In [18]:
pred

array([[[ 1.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [11.],
        [ 3.],
        [ 3.]]])

In [19]:
pred.shape

(1, 23, 1)

In [ ]:
# https://github.com/dyq0811/EEG-Transformer-seq2seq